In [ ]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact

proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"

raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)

GPT:

In [ ]:
bold_image = ants.image_read(bold_stim_filepath)  # Replace with your BOLD image path
t1_image = ants.image_read(t1_input_filepath)     # Replace with your T1 image path


# # If the BOLD image is 4D, extract the mean image for registration
# if bold_image.dimension == 4:
#     bold_mean_np = bold_image.numpy().mean(axis=-1)  # Compute the mean along the time axis
#     bold_mean = ants.from_numpy(bold_mean_np, spacing=bold_image.spacing[:3])
# else:
#     bold_mean = bold_image

# # Perform registration
# registration = ants.registration(
#     fixed=t1_image,
#     moving=bold_mean,
#     type_of_transform='Rigid'  # You can also use 'Affine' or 'SyN' for deformable registration
# )

In [ ]:
# print(f"BOLD image dimensions: {bold_image.dimension}, shape: {bold_image.shape}")
# print(f"T1 image dimensions: {t1_image.dimension}, shape: {t1_image.shape}")
# print(f"Forward Transforms: {registration['fwdtransforms']}")

In [ ]:
# bold_mean_registered = ants.apply_transforms(
#     fixed=t1_image,
#     moving=bold_mean,
#     transformlist=registration['fwdtransforms'],
#     interpolator='linear'
# )
# # #ants.image_write(bold_mean_registered, 'bold_mean_registered.nii.gz')


In [ ]:
# print(registration)

In [ ]:
# print(registration['fwdtransforms'])

In [ ]:
# print(bold_mean_registered)
# explore_3D_vol(bold_mean_registered.numpy())
# # explore_3D_vol(t1_image.numpy())

THIS WORKS!

The following does not (it crashes the kernel):

````
bold_registered = ants.apply_transforms(
    fixed=t1_image,
    moving=bold_image,
    transformlist=registration['fwdtransforms'],
    interpolator='linear',
    imagetype=3
)
````

I suspect this has to do with a glitch on the image type 3 as this was also an issue yesterday. Perhaps open an issue on github

As a workaround, let's use a loop to cycle through all the 4th dimensional stuff.

Given the computational lift of registration, let's write them to a list



In [ ]:
transforms = []
for frame in range(bold_image.shape[3]):
    print(f"frame {frame}/{bold_image.shape[3]}")
    print(" creating bold frame")
    bold_frame = ants.from_numpy(bold_image.numpy()[:,:,:,frame])
    print(" Registering bold frame to T1 image")
    registration = ants.registration(
    fixed=t1_image,
    moving=bold_frame,
    type_of_transform='Rigid'  # You can also use 'Affine' or 'SyN' for deformable registration
    )
    transforms += registration['fwdtransforms']